In [56]:
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import requests

In [99]:
webpage = 'https://medium.com/'
request = requests.get(webpage)
data = request.text
soup = BeautifulSoup(data, 'html.parser')

print(soup.prettify())

<!DOCTYPE html>
<html xmlns:cc="http://creativecommons.org/ns#">
 <head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# medium-com: http://ogp.me/ns/fb/medium-com#">
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   Medium – Read, write and share stories that matter
  </title>
  <link href="https://medium.com/" rel="canonical"/>
  <meta content="Medium – Read, write and share stories that matter" name="title"/>
  <meta content="unsafe-url" name="referrer"/>
  <meta content="Welcome to Medium, a place to read, write, and interact with the stories that matter most to you. Every day, thousands of voices read, write, and share important stories on Medium." name="description"/>
  <meta content="#000000" name="theme-color"/>
  <meta content="Medium – Read, write and share stories that matter" property="og:title"/>
  <meta content="https://medium.com/" property="og:url"/>
  <meta c

In [51]:
linklist = []
data = soup.findAll('div',attrs={'class':'js-homeStream'})
for div in data:
    links = div.findAll('a')
    for a in links:
        if a['href'][-1] != '-':
            continue
        else:
            trimmedlink = a['href'].split('?',1)[0]
            if trimmedlink in linklist:
                continue
            else:
                trimmedlink = a['href'].split('?',1)[0]
                linklist.append(trimmedlink)                

In [100]:
mediumlist = []
data = soup.findAll('div',attrs={'class':'js-homeStream'})
for div in data:
    links = div.findAll('a')
    for a in links:
        if a['href'][-1] != '-':
            continue
        elif 'https://medium.com' not in a['href']:
            continue
        else:
            trimmedlink = a['href'].split('?',1)[0]
            if trimmedlink in mediumlist:
                continue
            else:
                trimmedlink = a['href'].split('?',1)[0]
                mediumlist.append(trimmedlink)                

In [101]:
mediumlist

['https://medium.com/@tcolicchio/an-open-letter-to-male-chefs-742ca722e8f2',
 'https://medium.com/the-mission/how-to-tell-if-someone-is-truly-smart-or-just-average-a2f0bcac5db2',
 'https://medium.com/the-transition-transmission/the-transition-transmission-fdf7974cd3fa',
 'https://medium.com/@simon.sarris/after-universal-basic-income-the-flood-217db9889c07',
 'https://medium.com/@TransEquality/what-i-told-betsy-devos-as-the-mother-of-a-transgender-girl-39a8b59f8d60',
 'https://medium.com/intimates/the-fixer-upper-9dbf0f35d33e',
 'https://medium.com/@paulfidalgo/hearing-loss-b1f3111cb429',
 'https://medium.com/@shanisilver/and-these-are-just-the-famous-ones-d96cc68a1725',
 'https://medium.com/@forbes/a-first-look-into-alibabas-new-retail-vision-ahead-of-singles-day-103b480a943e',
 'https://medium.com/rollingstone/meet-ela-darling-porn-star-activist-tech-pioneer-1eef4cfe063e',
 'https://medium.com/mit-technology-review/reid-hoffman-its-time-to-change-silicon-valley-culture-6be7ead79f3e',


In [66]:
articles = ['https://hackernoon.com/looking-to-be-more-likeable-start-by-ditching-the-golden-rule-edfd7f930451', 
            'https://medium.com/personal-growth/the-ultimate-survival-guide-to-not-fitting-in-fe0660ffd6d7']

### Create Pandas Dataframe to store scraped data

In [62]:
cols = ['title', 'author', 'published_date', 'published_time', 'content']
article_data = pd.DataFrame(index =np.arange(len(linklist)), columns = cols)

### Web Scraping using BeautifulSoup and Requests

In [108]:
def mad_scraps(medium_page):
    cols = ['title', 'author', 'published_date', 'published_time', 'content']
    article_data = pd.DataFrame(columns = cols)
    length = len(medium_page)
    success_scrap = 0
    for i in range(0, length):
        article = medium_page[i]
        request = requests.get(article)
        data = request.text
        soup = BeautifulSoup(data, 'html.parser')
    
        title = soup.title.string.split(" – ",1)[0]
        author = soup.find(property='author')['content']
        if soup.find(property='article:published_time') != None:
            published_date = soup.find(property='article:published_time')['content'][:10]
        else:
            published_date = None
        #published_date = soup.find(property='article:published_time')['content'][:10]
        if soup.find(property='article:published_time') != None:
            published_time = soup.find(property='article:published_time')['content'][11:19]        
        else:
            published_time = None
        
        body = soup.findAll(["p","h2","h3","h4","h5","h6"])
        content = ''
        for x in range(0, len(body)-6):
            if isinstance(body[x].string, str):
                if body[x].name in ['h2','h3','h4','h5','h6']:
                    if content == '':
                        content += body[x].string + '\n'
                    else:
                        content += '\n \n' + body[x].string + '\n'
                if body[x].name == 'p':
                    content += body[x].string + ' '

        article_no = str(i+1)                    
                    
        if len(content) == 0:
            print('Did not manage to scrap article ' + article_no + ': Unable to scrap article content')
            continue
        
        data = [title, author, published_date, published_time, content]
        article_data.loc[success_scrap] = data
        
        success_scrap += 1
        
        print('Successfully scrapped article '+ article_no)
        
        time.sleep(0.5)
    
    percentage_scrapped = (success_scrap+1)/length
    print(str(percentage_scrapped))
    
    return article_data

In [109]:
mad_scraps(linklist)

Successfully scrapped article 1
Successfully scrapped article 2
Successfully scrapped article 3
Successfully scrapped article 4
Successfully scrapped article 5
Successfully scrapped article 6
Successfully scrapped article 7
Successfully scrapped article 8
Did not manage to scrap article 9: Unable to scrap article content
Successfully scrapped article 10
Did not manage to scrap article 11: Unable to scrap article content
Did not manage to scrap article 12: Unable to scrap article content
Successfully scrapped article 13
Did not manage to scrap article 14: Unable to scrap article content
Successfully scrapped article 15
Successfully scrapped article 16
Successfully scrapped article 17
Did not manage to scrap article 18: Unable to scrap article content
Did not manage to scrap article 19: Unable to scrap article content
Did not manage to scrap article 20: Unable to scrap article content
Did not manage to scrap article 21: Unable to scrap article content
Successfully scrapped article 22
Suc

,title,author,published_date,published_time,content
0,An Open Letter to (Male) Chefs,Tom Colicchio,2017-11-08,19:33:53,The recent ‘revelations’ of rampant harassment...
1,Woman Fired For Flipping Off Trump’s Motorcade...,Allan Ishac,2017-11-07,12:01:01,"Truth In Satire\n\n \nJuli Briskman, the 50-ye..."
2,How To Tell If Someone Is Truly Smart Or Just ...,Michael Simmons,2017-11-08,13:56:00,Have you ever noticed how some of the world’s ...
3,The Build Order Every Startup Should Follow To...,Wayne Chang,2017-11-07,21:11:54,The vast majority of startups follow a very si...
4,Avoid Burnout and Increase Awareness Using Ult...,Brad Buzzard,2017-11-07,17:01:01,Circadian rhythms’ lesser know cousins may hol...
5,Let’s talk about objectification,Kira Wertz,2017-08-05,15:16:43,Within a matter of days of coming out to my ol...
6,"After Universal Basic Income, The Flood",Simon Sarris,2017-10-22,14:48:24,What if we implement UBI and it makes everythi...
7,What I Told Betsy DeVos as the Mother of a Tra...,National Center for Transgender Equality,2017-10-27,17:50:16,"by Debi Jackson, Family Organizer I reached ou..."
8,Why Does America Have So Many Predators?,umair haque,2017-11-09,21:09:10,The Human Costs of a Predatory Society\nConsid...
9,The Paradise Papers Primer: Trump-Russia Links...,Remy Anne,2017-11-07,21:20:41,Leaked documents expose a secret world of hidd...


In [63]:
for i in range(0, len(linklist)):
    article = linklist[i]
    request = requests.get(article)
    data = request.text
    soup = BeautifulSoup(data, 'html.parser')
    
    title = soup.title.string.split(" – ",1)[0]
    author = soup.find(property='author')['content']
    published_date = soup.find(property='article:published_time')['content'][:10]
    published_time = soup.find(property='article:published_time')['content'][11:19]

    
    body = soup.findAll(["p","h2","h3","h4","h5","h6"])
    content = ''
    for x in range(0, len(body)-6):
        if isinstance(body[x].string, str):
            if body[x].name in ['h2','h3','h4','h5','h6']:
                if content == '':
                    content += body[x].string + '\n'
                else:
                    content += '\n \n' + body[x].string + '\n'
            if body[x].name == 'p':
                content += body[x].string + ' '
                
    data = [title, author, published_date, published_time, content]
    article_data.loc[i] = data
    
    time.sleep(1)

TypeError: 'NoneType' object is not subscriptable

In [10]:
article_data

,title,author,published_date,published_time,content
0,Looking To Be More Likeable? Start By Ditching...,Michael Thompson,2017-10-16,15:05:29,Forget what you learned as a kid. If you treat...
1,The Ultimate Survival Guide to Not Fitting In,Todd Brison,2017-10-17,11:09:12,Someone once told me to write about what I kno...


Save to csv file

In [11]:
dest = "Output/"
filename = 'multiple_articles_2 - ' + time.strftime("%d/%m/%Y") + '.csv'
article_data.to_csv(dest + filename, encoding='utf-8')

Save to excel file

In [12]:
dest = "Output/"
filename = 'multiple_articles_2 - ' + time.strftime("%d/%m/%Y") + '.xlsx'
article_data.to_excel(dest + filename, encoding='utf-8')